In [1]:
# script mdp4.py
from tkinter import *
import numpy as np

def initialize():
    global PosX,PosY,cost
# position initiale du robot
    PosX = 20+10*zoom
    PosY = 20+10*zoom
    for k in range(5):
        cost[k]=0
# cout et affichage
    Canevas.coords(Pion,PosX -9*zoom, PosY -9*zoom, PosX +9*zoom, PosY +9*zoom)
    wg.config(text=str(cost[1]))
    wb.config(text=str(cost[2]))
    wr.config(text=str(cost[3]))
    wn.config(text=str(cost[4]))
    ws.config(text='     total = '+str(cost[0]))


#deplace le robot selon la touche
def deplacement(touche,li,cj):
    haut(touche,li,cj)
    bas(touche,li,cj)
    droite(touche,li,cj)
    gauche(touche,li,cj)
    
    # on dessine le pion a sa nouvelle position
    Canevas.coords(Pion,PosX -9*zoom, PosY -9*zoom, PosX +9*zoom, PosY +9*zoom)
    cost[0]=0    
    for k in range(4):
        cost[0]+=cost[k+1]*weight[k+1]        
    wg.config(text=str(cost[1]))
    wb.config(text=str(cost[2]))
    wr.config(text=str(cost[3]))
    wn.config(text=str(cost[4]))
    ws.config(text='     total = '+str(cost[0]))


def haut(touche,li,cj):
    global PosX,PosY,cost,g
    # deplacement vers le haut
    if touche == 'a' and li>0 and g[li-1,cj,0]>0:
        PosY -= zoom*20
        cost[g[li-1,cj,0]]+=g[li-1,cj,1]  
    else:
        print('deplacement imposiible')

def bas(touche,li,cj):
    
    global PosX,PosY,cost,g
       # deplacement vers le bas
    if touche == 'q' and li<nblignes-1 and g[li+1,cj,0]>0:
        PosY += zoom*20
        cost[g[li+1,cj,0]]+=g[li+1,cj,1]
    else:
        print('deplacement imposiible')
        
def droite(touche,li,cj):
    global PosX,PosY,cost,g
    # deplacement vers la droite
    if touche == 'm' and cj< nbcolonnes-1 and g[li,cj+1,0]>0:
        PosX += zoom*20
        cost[g[li,cj+1,0]]+=g[li,cj+1,1] 
    else:
        print('deplacement imposiible')
        
def gauche(touche,li,cj):
    global PosX,PosY,cost,g
    # deplacement vers la gauche
    if touche == 'l' and cj >0 and g[li,cj-1,0]>0:
        PosX -= zoom*20
        cost[g[li,cj-1,0]]+=g[li,cj-1,1] 
    else:
        print('deplacement imposiible')
        
    
    

#pblanc correspond à la proba d'avoir des murs 
def colordraw(g,nblignes,nbcolonnes,pblanc,pverte,pbleue,prouge,pnoire):
    pblanc=pblanc
    pverte=pverte
    pbleue=pbleue
    prouge=prouge
    pnoire=pnoire
    # remplit la grille avec une couleur c selon les probas
    for i in range(nblignes):
        for j in range(nbcolonnes):
            z=np.random.uniform(0,1)
            if z < pblanc:
                c=0
            else:
                if z < pblanc+ pverte:
                    c=1
                else:
                    if z < pblanc+ pverte + pbleue:
                        c=2
                    else:
                        if z< pblanc+ pverte + pbleue +prouge:
                            c=3
                        else:
                            c=4   
            # si case non blanche on tire un chiffre entre 1 et 9
            if c>0:
                g[i,j,0]=c
                g[i,j,1]=np.random.random_integers(9)
    # pas de murs sur le robot et l'objectif initialialment
    g[0,0,0]=np.random.random_integers(3)
    g[0,1,0]=np.random.random_integers(3)
    g[2,0,0]=np.random.random_integers(3)     
    g[nblignes-1,nbcolonnes-1]=np.random.random_integers(3)
    g[nblignes-2,nbcolonnes-1]=np.random.random_integers(3)
    g[nblignes-1,nbcolonnes-2]=np.random.random_integers(3)
    for i in range(nblignes):
        for j in range(nbcolonnes):          
            y =zoom*20*i+20
            x =zoom*20*j+20
            #ecrit chiffre dans case colorés
            if g[i,j,0]>0:   
                #Canevas.create_oval(x+zoom*(10-3),y+zoom*(10-3),x+zoom*(10+3),y+zoom*(10+3),width=1,outline=color[g[i,j]],fill=color[g[i,j]])
                Canevas.create_text(x+zoom*(10),y+zoom*(10), text=str(g[i,j,1]),fill=color[g[i,j,0]],font = "Verdana "+str(int(6*zoom))+" bold")
            #mets des murs à la place des cases blanches
            else:
                Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=mywalls)
                

In [2]:


def Clavier(event):
    global PosX,PosY,cost,g
    touche = event.keysym
    #position du robot
    cj=(PosX-30)/(20*zoom)
    li=(PosY-30)/(20*zoom)
    li = int(li)
    cj = int(cj)
    deplacement(touche,li,cj)
    #résolution automatique

        
def deterministe(strategie):
    global PosX,PosY,cost,g
    for a in strategie:
        cj=int((PosX-30)/(20*zoom))
        li=int((PosY-30)/(20*zoom))
        deplacement(a,li,cj)

#strategie mixte lors de l'appuie sur le touche "espace"
def mixte():
    global PosX,PosY,cost,g
    cj=int((PosX-30)/(20*zoom))
    li=int((PosY-30)/(20*zoom))
    if np.random.uniform(0,1)<0.4:
            touche = 'm'
    else:
            if np.random.uniform(0,1)<0.8: 
                touche ='q'
            else:
                if np.random.uniform(0,1)<0.9: 
                    touche ='a'
                else:
                    touche = 'l'
    deplacement(touche,li,cj)

#deplce le robot avec espace selon si mixte est cohée ou pas
def deplacement_deterministe_mixte(event):
    touche = event.keysym
    if touche == 'space':
        print(check)
        if check:
            mixte()
        else:
            deterministe(['q'])
            

Mafenetre = Tk()
Mafenetre.title('MDP')

zoom=2

#taille de la grille
nblignes=10
nbcolonnes=20

# Creation d'un widget Canvas (pour la grille)
Largeur = zoom*20*nbcolonnes+40
Hauteur = zoom*20*nblignes+40
 
# valeurs de la grille
g= np.zeros((nblignes,nbcolonnes,2), dtype=np.int)
cost= np.zeros(5, dtype=np.int)
weight= np.zeros(5, dtype=np.int)
weight[1] = 1
weight[2] = 1
weight[3] = 1
weight[4] = 1

# def des couleurs
myred="#D20B18"
mygreen="#25A531"
myblue="#0B79F7"
mygrey="#E8E8EB"
myyellow="#F9FB70"
myblack="#2D2B2B"
mywalls="#5E5E64"
mywhite="#FFFFFF"
color=[mygrey,mygreen,myblue,myred,myblack]

# ecriture du quadrillage et coloration
Canevas = Canvas(Mafenetre, width = Largeur, height =Hauteur, bg =mywhite)
for i in range(nblignes+1):
    ni=zoom*20*i+20
    Canevas.create_line(20, ni, Largeur-20,ni)
for j in range(nbcolonnes+1):
    nj=zoom*20*j+20
    Canevas.create_line(nj, 20, nj, Hauteur-20)
colordraw(g,nblignes,nbcolonnes,0.2,0.2,0.2,0.2,0.2)

 
Canevas.focus_set()
Canevas.bind('<Key>',Clavier)
Canevas.bind('<Key>',deplacement)

Canevas.pack(padx =5, pady =5)

PosX = 20+10*zoom
PosY = 20+10*zoom

# Creation d'un widget Button (bouton Quitter)
Button(Mafenetre, text ='Restart', command = initialize).pack(side=LEFT,padx=5,pady=5)
Button(Mafenetre, text ='Quit', command = Mafenetre.destroy).pack(side=LEFT,padx=5,pady=5)

check = IntVar()
print(check)
c = Checkbutton(Mafenetre, text="Mixte", variable=check)
c.pack()

w = Label(Mafenetre, text='     Costs: ', fg=myblack,font = "Verdana "+str(int(5*zoom))+" bold")
w.pack(side=LEFT,padx=5,pady=5) 
wg = Label(Mafenetre, text=str(cost[1]),fg=mygreen,font = "Verdana "+str(int(5*zoom))+" bold")
wg.pack(side=LEFT,padx=5,pady=5) 
wb = Label(Mafenetre, text=str(cost[2]),fg=myblue,font = "Verdana "+str(int(5*zoom))+" bold")
wb.pack(side=LEFT,padx=5,pady=5) 
wr = Label(Mafenetre, text=str(cost[3]),fg=myred,font = "Verdana "+str(int(5*zoom))+" bold")
wr.pack(side=LEFT,padx=5,pady=5) 
wn = Label(Mafenetre, text=str(cost[4]),fg=myblack,font = "Verdana "+str(int(5*zoom))+" bold")
wn.pack(side=LEFT,padx=5,pady=5) 
ws = Label(Mafenetre, text='     total = '+str(cost[0]),fg=myblack,font = "Verdana "+str(int(5*zoom))+" bold")
ws.pack(side=LEFT,padx=5,pady=5) 
Pion = Canevas.create_oval(PosX-10,PosY-10,PosX+10,PosY+10,width=2,outline='black',fill=myyellow)


initialize()

Mafenetre.mainloop()

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:107: DeprecationWarning: This function is deprecated. Please call randint(1, 9 + 1) instead
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:109: DeprecationWarning: This function is deprecated. Please call randint(1, 3 + 1) instead
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:110: DeprecationWarning: This function is deprecated. Please call randint(1, 3 + 1) instead
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:111: DeprecationWarning: This function is deprecated. Please call randint(1, 3 + 1) instead
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:112: DeprecationWarning: This function is deprecated. Please call randint(1, 3 + 1) instead
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:113: DeprecationWarning: This function is deprecated. Please call randint(1, 3 + 1) instead
/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launc

PY_VAR0


In [76]:
def sum_p_v(val_etats,liste_actions,action,probas):
    somme = 0
    for i in range(len(val_etats)):
            for j in range(len(val_etats[0])):
                somme += probas[i][j][action]*val_etats[i][j]
    return somme

def iteration_de_la_valeur(nbLignes,nbCol,liste_actions,rewards,probas,gamma,epsilon):
    #2 etats :  - pour t-1 
    #           - pour t 
    val_etats =  np.zeros((nbLignes,nbCol))
    print(val_etats.shape)
    q = np.zeros((nbLignes,nbCol,len(liste_actions)))
    t = 0
    #critere d'arret
    vt, vt_prec = val_etats,val_etats
    while np.max(np.abs(vt - vt_prec)) >= epsilon or t==0:
        t += 1
        vt = val_etats
        for i in range(len(val_etats)):
            for j in range(len(val_etats[0])):
                for action in liste_actions:
                    q[i][j][action] = rewards[action] + gamma * sum_p_v(val_etats,liste_actions,action,probas)
                val_etats[i][j] = max(q[i][j])
        vt_prec = val_etats
    
    #politique optimal
    d = np.zeros((nbLignes,nbCol))
    for i in range(len(val_etats)):
            for j in range(len(val_etats[0])):
                d[i][j] = np.argmax(q[i][j])
    return d,val_etats,t 

rewards = [1,2,3,5]
probas = np.random.rand(2,3,4)
liste_actions = [0,1,2,3]
iteration_de_la_valeur(2,3,liste_actions,rewards,probas,0.1,1)

print(g.shape)

(2, 3)
(10, 20, 2)
